In [1]:
from bs4 import BeautifulSoup
from lxml import html
import urllib.request as Request
import csv
import time
import sys
import facebook
import re

def getXpath(node):
    expath = ""
    
    for parent in node.parents:
        expath = parent.name + "/" + expath
    
    expath = expath + node.name
    
    expath = expath.replace("[document]", "")
    
    return expath



In [2]:
def processFormat1(pagLxml, pagSoup):
    receta = []
    pagString = Request.urlopen(url).read()
    nodo1 = pagSoup.find_all("h2", class_="subbuzz_name") [0]
    titulo = pagLxml.xpath(getXpath(nodo1) + "/text()")[0]
    receta.append(titulo)
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_text buzz-superlist-item-text  buzz_superlist_item_wide   "]/div[@class="solid"]')
    nodo2 = nodos[0]
    nodoIngredients= nodo2.getchildren () [1:]
    ingreStr = ""
    for ig in nodoIngredients:
        ingreStr = ingreStr + ig.text_content().replace("\n","|") + "|"
    receta.append(ingreStr)
    nodoPreparacion = nodos[1].getchildren () [1:]
    prepStr = ""
    for prep in nodoPreparacion:
        prepStr = prepStr + prep.text_content ().replace("\n","|") + "|"
    receta.append(prepStr)
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_image buzz-superlist-item-image  buzz_superlist_item_wide image_hit  "]/div[@class="solid"]/div[@class="sub_buzz_content"]/img')
    receta.append(nodos[0].attrib["rel:bf_image_src"])
    nodos = pagLxml.xpath ('//div[@class="video-embed-big video-embed-area bf_dom"]')
    receta.append(nodos[0].attrib["rel:thumb"])
    receta.append(nodos[0].attrib["rel:bf_bucket_data"])
    return [receta]

In [3]:
def processFormat2(pagLxml, pagSoup):
    receta = []
    pagString = Request.urlopen(url).read()
    nodo1 = pagSoup.find_all("h2", class_="subbuzz_name") [0]
    titulo = pagLxml.xpath(getXpath(nodo1) + "/text()")[0]
    receta.append(titulo)
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_image buzz-superlist-item-image  buzz_superlist_item_wide image_hit  "]/div[@class="solid"]/p')
    ingreStr = nodos[0].text_content() + "|"
    ingreStr = ingreStr + nodos[1].text_content().replace("INGREDIENTS\n","").replace("\n","|")    
    receta.append(ingreStr)
    prepStr = nodos[2].text_content().replace("PREPARATION\n","").replace("\n","|")  
    receta.append(prepStr)
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_image buzz-superlist-item-image  buzz_superlist_item_wide image_hit  "]/div[@class="solid"]/div[@class="sub_buzz_content"]/img')
    receta.append(nodos[0].attrib["rel:bf_image_src"])
    nodos = pagLxml.xpath ('//div[@class="video-embed-big video-embed-area bf_dom"]')
    receta.append(nodos[0].attrib["rel:thumb"])
    receta.append(nodos[0].attrib["rel:bf_bucket_data"])
    return [receta]

In [4]:
def processFormat3(pagLxml):
    receta = []
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_embed buzz-superlist-item-embed  buzz_superlist_item_wide   "]/div[@class="sub_buzz_content solid"]/div[@itemtype="http://schema.org/Recipe"]')
    nodosIngConPrep = nodos [0].getchildren()[0].getchildren()
    titulo = ""
    ingredientes = ""
    preparacion = ""
    tituloEncontrado = False
    ingredientesEncontrado = False
    preparacionsEncontrado = False
    for n in nodosIngConPrep:
        texto = n.text_content()
        if(texto.strip() != ""):
            # "strip" es una funcion string= q busca espacio en 'blanco' al principio y final de X y los elimina.
            if(preparacionsEncontrado) :
                preparacion = preparacion + texto + "|"
            elif(ingredientesEncontrado ):
                if(texto.startswith("PREPARATION")):
                   # "startswith" compara si el 'comienzo' del texto es igual a la variable q se declaro.
                    preparacionsEncontrado = True
                else:
                    ingredientes = ingredientes + texto + "|"
            elif(tituloEncontrado):
                if(texto.startswith("INGREDIENTS")):
                    ingredientesEncontrado = True
            else:
                titulo = texto.replace("RECIPE: ", "")
                tituloEncontrado = True
    receta.append(titulo)
    receta.append(ingredientes)
    receta.append(preparacion)
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_image buzz-superlist-item-image  buzz_superlist_item_wide image_hit  "]/div[@class="solid"]/div[@class="sub_buzz_content"]/img')
    receta.append(nodos[0].attrib["rel:bf_image_src"])
    nodos = pagLxml.xpath ('//div[@class="video-embed-big video-embed-area bf_dom"]')
    receta.append(nodos[0].attrib["rel:thumb"])
    receta.append(nodos[0].attrib["rel:bf_bucket_data"])
    return [receta]

In [5]:
def processFormat4(pagLxml):
    nodos = pagLxml.xpath ('//div [@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_image buzz-superlist-item-image  buzz_superlist_item_wide image_hit  " ]/div [@class="solid"] ')
    titulo1 = nodos[1].getchildren ()[0].text_content ()[3:]
    titulo2 = nodos[2].getchildren ()[0].text_content ()[3:]
    titulo3 = nodos[3].getchildren ()[0].text_content ()[3:]
    titulo4 = nodos[4].getchildren ()[0].text_content ()[3:]
    ingredientes1 = nodos[1].getchildren ()[3].text_content ().replace("INGREDIENTS\n","")
    ingredientes2 = nodos[2].getchildren ()[3].text_content ().replace("INGREDIENTS\n","")
    ingredientes3 = nodos[3].getchildren ()[3].text_content ().replace("INGREDIENTS\n","")
    ingredientes4 = nodos[3].getchildren ()[3].text_content ().replace("INGREDIENTS\n","")
    preparacion1 = nodos[1].getchildren ()[4].text_content ().replace("PREPARATION\n","")
    preparacion2 = nodos[2].getchildren ()[4].text_content ().replace("PREPARATION\n","")
    preparacion3 = nodos[3].getchildren ()[4].text_content ().replace("PREPARATION\n","")
    preparacion4 = nodos[4].getchildren ()[4].text_content ().replace("PREPARATION\n","")
    nodos = pagLxml.xpath ('//div[@class="buzz_superlist_item buzz-superlist-item buzz_superlist_item_image buzz-superlist-item-image  buzz_superlist_item_wide image_hit  "]/div[@class="solid"]/div[@class="sub_buzz_content"]/img')
    imagen = nodos[0].attrib["rel:bf_image_src"]
    nodos = pagLxml.xpath ('//div[@class="video-embed-big video-embed-area bf_dom"]')
    video1 = nodos[0].attrib["rel:thumb"]
    video2 = nodos[0].attrib["rel:bf_bucket_data"]
    recetas = [ [titulo1, ingredientes1, preparacion1, imagen, video1, video2] ]
    recetas.append([titulo2, ingredientes2, preparacion2, imagen, video1, video2])
    recetas.append([titulo3, ingredientes3, preparacion3, imagen, video1, video2])
    recetas.append([titulo4, ingredientes4, preparacion4, imagen, video1, video2])
    return recetas

In [6]:
def processUrl(pagLxml, pagSoup, pageFormat):
    result = None
    if(pageFormat == 1):
        result = processFormat1(pagLxml, pagSoup)
    elif(pageFormat == 2):
        result = processFormat2(pagLxml, pagSoup)
    elif(pageFormat == 3):
        result = processFormat3(pagLxml)
    elif(pageFormat == 4):
        result = processFormat4(pagLxml)
    else:
        raise ValueError('Formato invalido')
    return result


In [7]:
def downloadUrlsFromTasty(url):
    urlTools = []
    pagString = Request.urlopen(url).read()
    pagLxml = html.fromstring (pagString)
    nodos = pagLxml.xpath('//div[@class="card xs-relative xs-mb1 js-feed-item"]/div[@class="card--video"]/div[@class="xs-p2 clearfix card-title"]/a[@class="xs-col-11 link-gray"]')
    for n in nodos:
        urlTools.append([n.find("h2").text_content(), n.attrib["href"]])
    return urlTools

In [8]:
def downloadUrlsFromFacebook():
    urlTools = []
    graph = facebook.GraphAPI(version='2.7')
    token = graph.get_app_access_token(app_id="1789388901276350", app_secret="a37bb243d30a1fd1fcef585447f7d323")
    graph = facebook.GraphAPI(access_token=token, version='2.7')
    #Hago la primera llamada al API de facebook y guardo los videos en una variable
    callRes = graph.get_object(id="buzzfeedtasty/videos", fields="description,id", type="uploaded", limit="100")
    videos = callRes["data"]
    #Reviso si la paginacion esta activa. Si lo esta es porque hay que seguir llamando al API
    while "paging" in callRes.keys():
        after = callRes["paging"]["cursors"]["after"]
        callRes = graph.get_object(id="buzzfeedtasty/videos", fields="description,id", type="uploaded", limit="100", after=after)
        videos = videos + callRes["data"]
    #Ahora me descargo el numero total de reacciones asociadas a cada video (esto incluye los likes)
    videosYReac = []
    sleepCounter = 50
    for v in videos:
        vCopy = v.copy()
        id = vCopy["id"] + "/reactions"
        callRes = graph.get_object(id=id, limit="0", summary="true")
        vCopy["reactions"] = callRes["summary"]["total_count"]
        videosYReac.append(vCopy)
        sleepCounter = sleepCounter - 1
        if(sleepCounter == 0):
            time.sleep( 3 )
            sleepCounter = 50
    #Ahora me descargo el numero total de comentarios asociadas a cada video (esto incluye los likes)
    videosYEstad = []
    sleepCounter = 50
    for v in videosYReac:
        vCopy = v.copy()
        id = vCopy["id"] + "/comments"
        callRes = graph.get_object(id=id, limit="0", summary="true")
        vCopy["comments"] = callRes["summary"]["total_count"]
        videosYEstad.append(vCopy)
        sleepCounter = sleepCounter - 1
        if(sleepCounter == 0):
            time.sleep( 3 )
            sleepCounter = 50
    #Ahora extraigo el resultado que es: nombre de la receta, link, reactions y comments
    singleReceiptRE = re.compile('(?P<name>^.+)\n+(FULL RECIPE: )(?P<link>http://bzfd.it/\S+)')
    multipleReceiptRE = re.compile('(?P<name>^.+)\n+(FULL RECIPES: )(?P<link>http://bzfd.it/\S+)')
    for v in videosYEstad:
        result = singleReceiptRE.match(v["description"])
        if result != None:
            name = result.group("name")
            shortLink = result.group("link")
            urlTools.append([name, shortLink, v["reactions"], v["comments"]])
        else:
            result = multipleReceiptRE.match(v["description"])
            if result != None:
                name = result.group("name")
                shortLink = result.group("link")
                urlTools.append([name, shortLink, v["reactions"], v["comments"]])
    return urlTools

In [9]:
def scrapUrls(urls):
    """Procesa todos los URLs por medio de un proceso de ensayo y error,
    y el resultado de cada procesamiento es guardado como una linea en el 
    archivo "tasty.csv".
    Primero se intenta con el formato 1, luego con el 3 y por ultimo con el 4.
    En caso de fallar tambien con el formato 4, el url y el error son 
    guardados en la lista de errores.
    La lista de errores es salvada en el archivo "tasty-errors.csv" y tambien 
    retornada como resultado de la funcion.
    """
    errors = []
    with open('tasty.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for l in urls:
            time.sleep( 2 )
            pagString = Request.urlopen(l[1]).read()
            pagLxml = html.fromstring (pagString)
            pagSoup = BeautifulSoup(pagString,"lxml")
            error = False
            try:
                recetas = processUrl(pagLxml, pagSoup, 1)
                for r in recetas:
                    writer.writerow(l + r)
            except:
                error = True

            if(error):
                try:
                    error = False
                    recetas = processUrl(pagLxml, pagSoup, 3)
                    for r in recetas:
                        writer.writerow(l + r)
                except:
                    error = True

            if(error):
                try:
                    recetas = processUrl(pagLxml, pagSoup, 4)
                    for r in recetas:
                        writer.writerow(l + r)
                except:
                    e = sys.exc_info()[0]
                    errors.append(l + [str(e)])

    with open('tasty-errors.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=';', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for e in errors:
            writer.writerow(e)
    return errors

In [10]:
useFacebook = True
if (useFacebook):
    links = downloadUrlsFromFacebook()
else:
    links = downloadUrlsFromTasty("https://www.buzzfeed.com/tasty/weeknights")
    links = links + downloadUrls("https://www.buzzfeed.com/tasty/cheatday")
    links = links + downloadUrls("https://www.buzzfeed.com/tasty/litebites")
    links = links + downloadUrls("https://www.buzzfeed.com/tasty/junior")
    links = links + downloadUrls("https://www.buzzfeed.com/tasty/vegetarian")
    links = links + downloadUrls("https://www.buzzfeed.com/tasty/happyhour")

In [11]:
print("El total de recetas es: " + str(len(links)))
for l in links[0:10]:
    print(l[0] + " --- " + l[1])

El total de recetas es: 327
Healthy Mediterranean Salad --- http://bzfd.it/2dABK48
Baked Parmesan Chicken Strips #TastyJunior --- http://bzfd.it/2da0hP8
Tornado Potato --- http://bzfd.it/2dxz475
These Cookie Butter Éclairs from Tasty Miam look so good and perfect for a delicious fall dessert. --- http://bzfd.it/2d6TTbi
3-Ingredient Chocolate Desserts  --- http://bzfd.it/2duqpCB
Here's a tasty One-Pot Hot Dog Pasta from Tasty Demais that is so good you have to try it!  --- http://bzfd.it/2duga19
Slow Cooker Short Ribs --- http://bzfd.it/2daLmpB
Banana French Toast Bake --- http://bzfd.it/2cty4DC
We are sooooo loving our friends at  Tasty Japan for these adorable animal-shaped boiled eggs. --- http://bzfd.it/2dkWoor
4 Easy 3-Ingredient Dinners --- http://bzfd.it/2dkUaFW


In [12]:
errors = scrapUrls(links)
print(str(len(errors)))

287


In [ ]:
with open('tasty.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';', quotechar='|')
    i = 1
    for row in reader:
        print(str(i) + " --- " + row[0])
        i = i + 1

In [ ]:
for e in errors:
    #print(e[0] + " --- " + e[1] + " --- " + e[2])
    print(e[1])

In [13]:
url = "https://www.buzzfeed.com/hannahwilliams/blueberry-peach-slab-pie?bffbtasty&ref=bffbtasty&utm_term=.jvewrXGVE#.smElQ1qL6"
url = "http://bzfd.it/2dABK48"
pagString = Request.urlopen(url).read()
pagLxml = html.fromstring (pagString)
pagSoup = BeautifulSoup(pagString,"lxml")

print(processUrl(pagLxml, pagSoup, 1))

[[' Healthy Mediterranean Salad', '1 chicken breast|1 tablespoon olive oil|1 tablespoon lemon juice|2 cloves garlic, minced|1 tablespoon fresh dill|salt and pepper, to taste|Dressing|1/3 cup greek yogurt|2 tablespoons olive oil|1 tablespoon red wine vinegar|1 tablespoon lemon juice|1 tablespoon water |1 clove garlic, minced|1 tablespoon fresh dill|salt and pepper, to taste|Toppings|1 head romaine lettuce, chopped|1/2 cup cucumber, sliced and halved|1/2 cup olives, halved|1/2 cup red onion, thinly sliced|1/2 cup cherry or grape tomatoes, halved|1/2 cup feta, crumbled|', '1. Preheat oven to 400°F/200°C. Place chicken breast on a baking sheet lined with aluminum foil or parchment paper. Top with olive oil, lemon juice, garlic, dill, and salt and pepper to season. |2. Bake for 35-40 minutes until internal temperature reaches 165°F/75°C and the juices run clear. |3. Slice thinly or cube chicken, cutting to your preference for topping the salad.|4. Add all dressing ingredients in a small bow

In [ ]:
url = "https://www.buzzfeed.com/alvinzhou/heres-four-ways-you-can-enjoy-rotini-pasta-this-week-for-lun?bffbtasty&ref=bffbtasty&utm_term=4ldradw#4ldradw"
pagString = Request.urlopen(url).read()
pagLxml = html.fromstring (pagString)
pagSoup = BeautifulSoup(pagString,"lxml")

print(processUrl(pagLxml, pagSoup, 4))

In [ ]:
url = "http://www.buzzfeed.com/caroltan/this-is-how-you-make-black-bean-burgers#.ybAdbGBYN"
pagString = Request.urlopen(url).read()
pagLxml = html.fromstring (pagString)
pagSoup = BeautifulSoup(pagString,"lxml")

print(processUrl(pagLxml, pagSoup, 3))

In [ ]:
url = "https://www.buzzfeed.com/camillebergerson/these-amazing-no-bake-desserts-have-only-three-ingredients-s?bffbtasty&ref=bffbtasty&utm_term=.jgkbK51yK#.ajmWYVNZY"
pagString = Request.urlopen(url).read()
pagLxml = html.fromstring (pagString)
pagSoup = BeautifulSoup(pagString,"lxml")

print(processUrl(pagLxml, pagSoup, 4))

In [ ]:
links = []
links.append(["Formato2", "https://www.buzzfeed.com/alvinzhou/heres-four-ways-you-can-enjoy-rotini-pasta-this-week-for-lun?bffbtasty&ref=bffbtasty&utm_term=4ldradw#4ldradw"])
links.append(["Formato3", "http://www.buzzfeed.com/caroltan/this-is-how-you-make-black-bean-burgers#.ybAdbGBYN"])
links.append(["Formato4", "https://www.buzzfeed.com/camillebergerson/these-amazing-no-bake-desserts-have-only-three-ingredients-s?bffbtasty&ref=bffbtasty&utm_term=4ldradw#4ldradw"])
links.append(["Formato1", "https://www.buzzfeed.com/hannahwilliams/blueberry-peach-slab-pie?bffbtasty&ref=bffbtasty&utm_term=.jvewrXGVE#.smElQ1qL6"])
errors = scrapUrls(links)
print("Total de errores: " + str(len(errors)))
for e in errors:
    print(e[0] + " --- " + e[1] + " --- " + e[2])
print("\n\nContenido de tasty.csv:\n")
with open('tasty.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';', quotechar='|')
    i = 1
    for row in reader:
        print(str(i) + " --- " + row[0])
        i = i + 1